In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['MD_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2520)

In [15]:
x[:,:2]

matrix([[  638717.30458734,   639265.7632429 ],
        [  677226.07891317,   677047.2798253 ],
        [ 1201092.48317194,  1206589.90847691],
        [ 1238377.97511772,  1264408.99515883],
        [  324342.0945821 ,   315812.37518491],
        [  402316.51685467,   396726.80484764],
        [  786270.61263762,   766612.07577184],
        [  779845.97374637,   782186.99964509],
        [  730965.96926564,   728562.67052238],
        [  743465.02563257,   731694.26725741],
        [  763821.42476145,   737182.55198116],
        [  643928.53578403,   662958.85970479],
        [  184587.57818664,   171665.08096128],
        [  187662.86284534,   185937.19043929],
        [  289492.02033941,   295412.65662866],
        [  260936.98240509,   266527.34250996],
        [  403671.48329635,   410312.75856755],
        [  405872.08533654,   412976.13427638],
        [  652967.70630269,   652046.65250106],
        [  604919.97682616,   589420.06186834],
        [ 1007412.11155524,  1001545.733

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [9e-02, 4e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.33200271e+07  0.00000000e+00  5.92e+03 2.77e+03  1.00e+06     0s
   1  -4.30227487e+07 -1.17021225e+07  2.07e+03 9.69e+02  3.63e+05     0s
   2  -3.83730178e+07 -4.50817066e+07  7.36e+01 3.45e+01  1.71e+05     0s
   3  -3.90503854e+07 -4.00361780e+07  7.36e-05 3.45e-05  1.7

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_MD.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[7.24659319171314e-11,
 478818.35086736834,
 315798.53580496024,
 3.267717941727303e-08,
 193395.0774002348,
 4.666379833339734e-08,
 5.914775723782177e-08,
 2.3996807324873307e-10,
 4.481294618606095e-12,
 353777.328797801,
 4.4806096371328195e-12,
 4.481286292886818e-12,
 4.480838683815012e-12,
 4.511037427103197e-12,
 212273.8222353439,
 8.252500170731958e-12,
 2.628666372835312e-09,
 3.1972925165467685e-08,
 599216.9587146047,
 577031.1174528877,
 5.610147631814646e-08,
 236316.9975221075,
 555171.7642079358,
 6.244180993665154e-09,
 77395.99152915218,
 41022.896571824094,
 6.495582754935695e-08,
 365687.4887320606,
 800983.733773289,
 8.252473728540484e-12,
 5.369328052857925e-09,
 130752.46046628794,
 198585.6772597659,
 2.9392976443080416e-09,
 191008.78615581978,
 637419.7461524124,
 8.252473614588083e-12,
 5.519647047952232e-09,
 8.508917981305344e-08,
 366354.26614050753,
 742431.6162563949,
 21139.79197624337,
 1.8230163055090527e-08,
 8.248837430335354e-12,
 4.5755993372241